In [12]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

In [61]:
JLPT = pd.read_csv('data/Core Japanese Vocabulary Extended.txt', 
                   sep='\t', header=None, 
                   names=['jp', 'en', 'jp_kana', 'pos', 'more_en', 'sound', 'level'])[['jp', 'en', 
                                                    'jp_kana', 'pos', 'more_en', 'level']]

JLPT.level = JLPT.level.fillna('jlpt-n0').apply(lambda s: s[-7:]) # The last level marker is always the lowest
JLPT.pos = JLPT.pos.fillna('None') # These mostly seem like interjections

JLPT.sample(20)

,jp,en,jp_kana,pos,more_en,level
4543,認識,"recognition, awareness, perception, understand...",認識[にんしき],"Noun, Suru verb",NaN,jlpt-n1
4165,ニコニコ,"with a friendly grin, smilingly",ニコニコ,"Adverb, Adverb taking the 'to' particle, Suru ...",NaN,jlpt-n2
11705,クッキー,"cookie, biscuit",クッキー,Noun,cookie (browser-related file sent from a WWW s...,jlpt-n0
14941,物件,"article, thing, object, property (real estate)",物件[ぶっけん],Noun,NaN,jlpt-n0
4133,半径,radius,半径[はんけい],Noun,NaN,jlpt-n2
17036,パルサー,pulsar,パルサー,Noun,NaN,jlpt-n0
11826,ダウンロード,download,ダウンロード,"Noun, Suru verb",NaN,jlpt-n0
3746,妥当,"valid, proper, right, appropriate, reasonable",妥当[だとう],"Na-adjective, Noun",NaN,jlpt-n2
8763,平静,"calm, serenity, tranquillity, tranquility",平静[へいせい],"Na-adjective, Noun",NaN,jlpt-n0
366,細い,"thin, slender, fine",細[ほそ]い,I-adjective,unlucky,jlpt-n5


In [67]:
JLPT[JLPT.pos.fillna('').str.contains('Suru verb')].sample(20)

,jp,en,jp_kana,pos,more_en,level
4467,勝利,"victory, triumph, conquest, success, win",勝利[しょうり],"Noun, Suru verb",NaN,jlpt-n1
8726,同じく,"similarly, same (idea), same (name)",同[おな]じく,"Adverb, Conjunction, Suru verb",NaN,jlpt-n0
7671,調達,"supply, provision, raising, procurement",調達[ちょうたつ],"Noun, Suru verb",NaN,jlpt-n0
9433,緊迫,"tension, strain",緊迫[きんぱく],"Noun, Suru verb",NaN,jlpt-n0
1991,解釈,"explanation, interpretation",解釈[かいしゃく],"Noun, Suru verb",NaN,jlpt-n3
5296,分担,"taking on one's share (e.g. of work), dividing...",分担[ぶんたん],"Noun, Suru verb, No-adjective",NaN,jlpt-n1
1666,議論,"argument, discussion, dispute, controversy",議論[ぎろん],"Noun, Suru verb",NaN,jlpt-n3
9642,保全,"preservation, integrity, conservation, mainten...",保全[ほぜん],"Noun, Suru verb",NaN,jlpt-n0
17447,スワップ,swap,スワップ,"Noun, Suru verb",NaN,jlpt-n0
10560,添付,"attaching (documents, etc.), appending, affixing",添付[てんぷ],"Noun, Suru verb, No-adjective",NaN,jlpt-n0
